<a href="https://colab.research.google.com/github/HajarahM/Mistral-7B/blob/main/Chat_with_MultiplePDFs_Mistral_7B_Instruct1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain
%pip install torch
%pip install sentence_transformers
%pip install faiss-cpu
%pip install huggingface-hub
%pip install pypdf
%pip -q install accelerate
%pip install llama-cpp-python
%pip -q install git+https://github.com/huggingface/transformers


In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFium2Loader
import ocrmypdf

In [ ]:
#load pdf files
loader = DirectoryLoader("./Laws/Petroleum")
data = loader.load()

In [ ]:
print(data)

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100, separators=['\n\n', '\n', '.'])

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

In [ ]:
#get the third chunk
text_chunks[400]

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = OllamaEmbeddings(model="mistral")

In [20]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_model = SentenceTransformerEmbeddings(model_name='BAAI/bge-large-zh-v1.5')

In [21]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(text_chunks, embedding=embedding_model)
print('saved embeddings to vector_store')

saved embeddings to vector_store


In [29]:
#Step 08: Create Embeddings for each of the Text Chunk
import os
vector_store = FAISS.from_documents(text_chunks, embedding=embedding_model)
print('saving embeddings to vector_store')
folder_path="vector_store"
if os.path.exists(folder_path):
    faiss_index=FAISS.load_local(folder_path, embeddings)
    faiss_index.merge_from(vector_store)
    faiss_index.save_local(folder_path)
else:
    vector_store.save_local(folder_path)

saving embeddings to vector_store


In [30]:
#Import Model
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(
    base_url="http://localhost:11434",
    model = "mistral",
    verbose=True,
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    )

In [43]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_rerank", retriever=vector_store.as_retriever(search_kwargs={"k": 4}))

In [44]:
query = "what are the national content requirements"

In [45]:
qa.run(query)

/opt/miniconda3/envs/data_processor/lib/python3.9/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



The national content requirements refer to regulations set forth in the Petroleum (Exploration, Development and Production) Act, 2013, Act No. 3 of 2013, which require companies involved in midstream operations in Uganda to comply with certain criteria related to the provision of goods and services for those operations. These requirements include a national supplier database that lists qualified Ugandan companies and registered entities, as well as qualification criteria and annual review processes. The national content requirements are designed to ensure that local businesses have opportunities to participate in the midstream sector and contribute to economic development in Uganda.

Score: 100
The national content requirements refer to the specific obligations that a foreign oil and gas project, such as the EACOP project, must fulfill in order to meet the content requirements of the host country, Uganda. These requirements are intended to ensure that the benefits of the project are s

ValueError: Could not parse output: 
The national content requirements refer to the specific obligations that a foreign oil and gas project, such as the EACOP project, must fulfill in order to meet the content requirements of the host country, Uganda. These requirements are intended to ensure that the benefits of the project are shared with the local population through employment, training, supply of goods and services, and transfer of technology.

The national content requirements for the EACOP project are set out in the Host Government Agreement and include specific targets and deadlines for meeting certain objectives related to national content development. These requirements include obligations for the EACOP project to provide Ugandans with employment and training opportunities, as well as to source goods and services from Ugandan suppliers where possible.

The national content requirements also include provisions for the creation of a national supplier data base that will facilita

In [46]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

/opt/miniconda3/envs/data_processor/lib/python3.9/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



The regulation says that licensees, contractors, subcontractors, and other entities engaged in midstream operations must comply with national content policies. The licensee is responsible for communicating their policy to their contractors and subcontractors and ensuring that they comply with it. Contractors and subcontractors are required to report national content information to the licensee and the Authority upon request. The Authority may determine, where possible, the fabrication and welding activities that shall be undertaken in Uganda by these entities, and shall also develop a national supplier database of qualified Ugandan companies and registered entities as well as qualified foreign companies registered in accordance with the Companies Act, 2012. The Authority also establishes and operates a national human capacity register to monitor and maintain a database of available human capabilities and technical skills for recruitment purposes. Failure to comply with these requireme

KeyboardInterrupt: 